<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0


    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-01 12:21:11
-------------------
qualified stocks: 91
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.28 L
Current:  1.38 C
-------------------
Today PnL: 1.14 L (0.84%)
Current PnL: -19.57 L (-13.16%)
CY Booked + Current PnL: -10.53 L (-7.08%)
-------------------
Total profit:  2.98 L
Total loss:  -22.55 L
-------------------
Total Booked + Current PnL: 16.45 L (13.52%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.57%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 80.95 L (58.8%)
Deployed:  1.22 C
Current:  1.38 C
CAGR/XIRR %: 7.81%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-17.42,60.0,H-LC,7.90,114080.0,9768.0,6879.0,1.29,9.36,6.03,15.96,9.0,1.42,0.86,27.84,X5K,ATH,METALS
53,MEDANTA,1486.00,12.31,57.0,H-MC,15.08,160358.0,34158.0,12011.0,0.61,27.07,7.49,36.59,108.0,2.84,1.20,43.02,XY24,NTT,HEALTHCARE
81,TTKPRESTIG,770.00,103.84,67.0,M-SC,7.66,87855.0,-12922.0,13011.0,2.12,-12.82,14.81,0.09,245.0,-0.99,0.66,15.42,OX40N,NTT,DURABLES
26,DIXON,18940.15,-31.26,63.0,H-MC,6.54,137464.0,22528.0,14063.0,2.95,19.60,10.23,31.83,81.0,1.60,1.03,42.48,X40N,ATH,IT
4,APOLLOHOSP,8285.00,-15.85,54.0,H-LC,8.46,168190.0,12013.0,14078.0,0.46,7.69,8.37,16.71,22.0,0.85,1.26,26.47,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,ICICIPRULI,790.00,-20.54,39.0,H-MC,2.25,136741.0,953.0,41802.0,0.87,0.70,30.57,31.48,107.0,0.02,1.03,12.91,X40,ATH,INSURANCE
38,ICICIGI,2260.25,-20.81,35.0,H-MC,4.76,136145.0,12.0,31109.0,0.01,0.01,22.85,22.86,91.0,0.00,1.02,12.62,X40,ATH,INSURANCE
54,NATIONALUM,247.44,-49.46,53.0,H-MC,1.38,100854.0,350.0,30286.0,2.21,0.35,30.03,30.49,79.0,0.01,0.76,32.87,MH,ATH,METALS


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VBL,671.64,-14.62,38.0,H-LC,7.82,307293.0,-8549.0,121227.0,-1.13,-2.71,39.45,35.67,5.0,-0.07,2.30,10.77,X40N,ATH,FMCG
1,ABB,7934.00,-41.52,40.0,H-LC,13.70,244320.0,-17299.0,136502.0,1.82,-6.61,55.87,45.57,7.0,-0.13,1.83,3.96,AR,NTT,ELECTRICAL
47,JIOFIN,387.00,-1.07,41.0,H-LC,14.94,218785.0,1379.0,52115.0,0.27,0.63,23.82,24.61,48.0,0.03,1.64,55.80,XY24,BTT,FINANCE
33,HCLTECH,1943.91,1.08,42.0,H-LC,10.05,230539.0,-11385.0,74649.0,0.93,-4.71,32.38,26.15,8.0,-0.15,1.73,8.94,X40,ATH,IT
84,UNITDSPR,1644.00,-13.80,47.0,H-LC,8.69,173382.0,-8377.0,43623.0,0.19,-4.61,25.16,19.39,86.0,-0.19,1.30,3.16,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,NESTLEIND,1377.0,-11.15,55.0,H-LC,2.67,273499.0,8073.0,48710.0,1.09,3.04,17.81,21.40,11.0,0.17,2.05,9.89,XY25,NTT,FMCG
51,LTIM,7230.2,-6.07,52.0,H-LC,2.69,232965.0,-13186.0,92394.0,0.85,-5.36,39.66,32.18,16.0,-0.14,1.75,28.76,X200,ATH,IT
45,ITC,452.0,-37.92,51.0,H-LC,2.98,200368.0,230.0,20658.0,2.21,0.11,10.31,10.44,4.0,0.01,1.50,5.68,X40,NTT,FMCG
20,CIPLA,1795.0,-18.41,62.0,H-LC,4.71,217433.0,12933.0,28484.0,-0.14,6.32,13.10,20.25,10.0,0.45,1.63,15.61,X40N,BTT,PHARMA
5,ASIANPAINT,4250.0,-10.58,64.0,H-LC,7.48,225773.0,-25995.0,148220.0,1.87,-10.32,65.65,48.55,27.0,-0.18,1.69,21.44,X40,BTT,PAINTS


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,VAIBHAVGBL,521.00,44.35,51.0,H-SC,11.84,96226.0,-55765.0,129886.0,8.55,-36.69,134.98,48.77,125.0,-0.43,0.72,12.00,XR,NTT,JEWELLERY
36,HINDZINC,730.22,17.01,56.0,M-LC,3.60,190878.0,-14198.0,126762.0,4.66,-6.92,66.41,54.89,52.0,-0.11,1.43,15.50,X5K,ATH,METALS
26,DIXON,18940.15,-31.26,63.0,H-MC,6.54,137464.0,22528.0,14063.0,2.95,19.60,10.23,31.83,81.0,1.60,1.03,42.48,X40N,ATH,IT
18,CAMS,5211.76,-6.49,47.0,H-SC,5.87,106431.0,4427.0,39497.0,2.59,4.34,37.11,43.06,122.0,0.11,0.80,22.20,X40N,ATH,MISC
88,VOLTAS,1918.49,0.60,65.0,H-MC,5.92,211125.0,19383.0,76638.0,2.44,10.11,36.30,50.08,99.0,0.25,1.58,17.59,XY25,ATH,AC


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-37.63,67.0,L-SC,29.29,91277.0,-22272.0,62324.0,-4.98,-19.61,68.28,35.27,268.0,-0.36,0.68,123.40,XR,NTT,HOTELS
87,VBL,671.64,-14.62,38.0,H-LC,7.82,307293.0,-8549.0,121227.0,-1.13,-2.71,39.45,35.67,5.0,-0.07,2.30,10.77,X40N,ATH,FMCG
50,LAOPALA,464.00,105.71,40.0,H-SC,2.79,65422.0,-35424.0,61248.0,-1.12,-35.13,93.62,25.61,142.0,-0.58,0.49,21.29,AR,NTT,CERAMICS
82,UJJIVANSFB,60.00,80.86,39.0,H-SC,14.31,113994.0,-28485.0,48003.0,-1.08,-19.99,42.11,13.70,163.0,-0.59,0.85,34.54,OX40N,NTT,BANKS
41,INDIGOPNTS,1408.00,105.92,37.0,M-SC,2.49,134900.0,-39659.0,39688.0,-1.07,-22.72,29.42,0.02,221.0,-1.00,1.01,17.34,OX40N,NTT,PAINTS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TTKPRESTIG,770.0,103.84,67.0,M-SC,7.66,87855.0,-12922.0,13011.0,2.12,-12.82,14.81,0.09,245.0,-0.99,0.66,15.42,OX40N,NTT,DURABLES
48,KANSAINER,340.0,-68.70,50.0,H-SC,3.56,216189.0,-53478.0,89805.0,0.09,-19.83,41.54,13.47,138.0,-0.60,1.62,9.35,XY24,NTT,PAINTS
82,UJJIVANSFB,60.0,80.86,39.0,H-SC,14.31,113994.0,-28485.0,48003.0,-1.08,-19.99,42.11,13.70,163.0,-0.59,0.85,34.54,OX40N,NTT,BANKS
19,CERA,9475.0,-20.76,41.0,H-SC,3.06,113112.0,-30691.0,57438.0,-0.43,-21.34,50.78,18.60,149.0,-0.53,0.85,24.82,OX40N,NTT,CERAMICS
41,INDIGOPNTS,1408.0,105.92,37.0,M-SC,2.49,134900.0,-39659.0,39688.0,-1.07,-22.72,29.42,0.02,221.0,-1.00,1.01,17.34,OX40N,NTT,PAINTS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TCS,4998.00,-26.44,51.0,H-LC,15.04,293393.0,-52563.0,176417.0,1.18,-15.19,60.13,35.80,1.0,-0.30,2.20,3.94,X40,BTT,IT
43,INFY,2275.00,-18.16,50.0,H-LC,11.57,318669.0,5781.0,165899.0,1.80,1.85,52.06,54.87,3.0,0.03,2.39,8.62,X40,BTT,IT
45,ITC,452.00,-37.92,51.0,H-LC,2.98,200368.0,230.0,20658.0,2.21,0.11,10.31,10.44,4.0,0.01,1.50,5.68,X40,NTT,FMCG
87,VBL,671.64,-14.62,38.0,H-LC,7.82,307293.0,-8549.0,121227.0,-1.13,-2.71,39.45,35.67,5.0,-0.07,2.30,10.77,X40N,ATH,FMCG
1,ABB,7934.00,-41.52,40.0,H-LC,13.70,244320.0,-17299.0,136502.0,1.82,-6.61,55.87,45.57,7.0,-0.13,1.83,3.96,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.0,12.31,57.0,H-MC,15.08,160358.0,34158.0,12011.0,0.61,27.07,7.49,36.59,108.0,2.84,1.20,43.02,XY24,NTT,HEALTHCARE
27,DMART,5201.0,-6.13,75.0,H-LC,13.46,234328.0,49847.0,20527.0,0.56,27.02,8.76,38.14,38.0,2.43,1.76,40.49,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,12.31,57.0,H-MC,15.08,160358.0,34158.0,12011.0,0.61,27.07,7.49,36.59,108.0,2.84,1.20,43.02,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,11.32,63.0,H-MC,3.76,192050.0,27380.0,40580.0,2.06,16.63,21.13,41.28,89.0,0.67,1.44,25.94,X40N,ATH,AC
88,VOLTAS,1918.49,0.60,65.0,H-MC,5.92,211125.0,19383.0,76638.0,2.44,10.11,36.30,50.08,99.0,0.25,1.58,17.59,XY25,ATH,AC
34,HEROMOTOCO,5949.07,-2.52,73.0,H-MC,17.71,180985.0,30072.0,27238.0,1.64,19.93,15.05,37.97,93.0,1.10,1.36,50.27,AR,ATH,AUTO
24,DABUR,735.00,-6.36,55.0,H-MC,3.72,203951.0,6923.0,82702.0,0.36,3.51,40.55,45.49,102.0,0.08,1.53,17.01,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,-2.52,73.0,H-MC,17.71,180985.0,30072.0,27238.0,1.64,19.93,15.05,37.97,93.0,1.10,1.36,50.27,AR,ATH,AUTO
40,INDIAMART,4850.92,-50.72,52.0,H-SC,12.52,137842.0,14506.0,119261.0,0.14,11.76,86.52,108.45,119.0,0.12,1.03,36.74,AR,ATH,MISC
89,WHIRLPOOL,2270.00,-42.68,47.0,M-SC,5.57,96952.0,5454.0,73296.0,-0.61,5.96,75.60,86.07,223.0,0.07,0.73,40.79,XR,NTT,DURABLES
90,WIPRO,420.00,-11.52,53.0,M-LC,7.57,155794.0,4849.0,104600.0,0.75,3.21,67.14,72.51,53.0,0.05,1.17,9.43,XR,NTT,IT
54,NATIONALUM,247.44,-49.46,53.0,H-MC,1.38,100854.0,350.0,30286.0,2.21,0.35,30.03,30.49,79.0,0.01,0.76,32.87,MH,ATH,METALS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-6.36,38.0,H-SC,16.46,87118.0,-13845.0,108375.0,1.02,-13.71,124.40,93.63,148.0,-0.13,0.65,29.76,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-37.63,67.0,L-SC,29.29,91277.0,-22272.0,62324.0,-4.98,-19.61,68.28,35.27,268.0,-0.36,0.68,123.40,XR,NTT,HOTELS
52,MASFIN,398.61,-19.15,47.0,H-SC,12.88,92115.0,-5865.0,27469.0,-0.98,-5.99,29.82,22.05,152.0,-0.21,0.69,33.90,XR,ATH,FINANCE
82,UJJIVANSFB,60.00,80.86,39.0,H-SC,14.31,113994.0,-28485.0,48003.0,-1.08,-19.99,42.11,13.70,163.0,-0.59,0.85,34.54,OX40N,NTT,BANKS
35,HINDALCO,756.01,-17.42,60.0,H-LC,7.90,114080.0,9768.0,6879.0,1.29,9.36,6.03,15.96,9.0,1.42,0.86,27.84,X5K,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-37.63,67.0,L-SC,29.29,91277.0,-22272.0,62324.0,-4.98,-19.61,68.28,35.27,268.0,-0.36,0.68,123.40,XR,NTT,HOTELS
34,HEROMOTOCO,5949.07,-2.52,73.0,H-MC,17.71,180985.0,30072.0,27238.0,1.64,19.93,15.05,37.97,93.0,1.10,1.36,50.27,AR,ATH,AUTO
27,DMART,5201.00,-6.13,75.0,H-LC,13.46,234328.0,49847.0,20527.0,0.56,27.02,8.76,38.14,38.0,2.43,1.76,40.49,X40N,NTT,FMCG
14,BLUESTARCO,2326.38,11.32,63.0,H-MC,3.76,192050.0,27380.0,40580.0,2.06,16.63,21.13,41.28,89.0,0.67,1.44,25.94,X40N,ATH,AC
60,RELAXO,1176.00,-33.59,58.0,H-SC,7.93,61606.0,-53932.0,88922.0,0.77,-46.68,144.34,30.28,136.0,-0.61,0.46,21.12,X40N,NTT,FOOTWEAR


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.68
1,25,43.13
2,50,72.73


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    36.94
LC    36.81
MC    26.26
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.10
X40      15.56
X40N     14.22
XY25      9.79
AR        9.31
XR        8.96
OX40N     7.28
X5K       2.29
X200      1.75
MH        1.68
SR        1.07
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.63
H-MC    23.40
H-SC    22.22
M-SC    13.14
M-LC     7.14
M-MC     2.54
L-SC     1.58
L-LC     1.04
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.14,-1.16,34.12
IT,13.40,-15.55,72.86
BANKS,7.26,-17.81,67.83
FINANCE,7.20,-25.26,71.45
PAINTS,5.95,-16.16,42.15
MISC,5.85,-17.78,85.30
ELECTRICAL,5.69,-8.51,68.31
AUTO,4.57,-11.10,59.43
AC,3.75,3.18,29.61


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2888734.0,23
AR,1237960.0,10
XR,1099604.0,13
X40,892770.0,10
X40N,536057.0,11
OX40N,486059.0,10
XY25,435294.0,7
SR,200399.0,2
X5K,133641.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2810670.0,27
M-SC,1494077.0,17
H-LC,1412664.0,17
H-MC,1256789.0,18
M-LC,427111.0,5
M-MC,338042.0,2
L-SC,250668.0,3
L-MC,60028.0,1
L-LC,45057.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,950481.0,6
M-SC,XY24,875340.0,7
H-SC,AR,670158.0,5
H-LC,X40,585843.0,5
H-SC,XR,530056.0,6
H-MC,XY24,492863.0,5
H-LC,AR,379160.0,2
M-MC,XY24,338042.0,2
H-LC,X40N,227939.0,5


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
